In [1]:
import tensorflow as tf
from tqdm import tqdm, trange
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.models import load_model, Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Softmax, Input, Reshape,Lambda,\
    Conv2D, MaxPool2D, Dropout,Concatenate,Embedding,BatchNormalization,TimeDistributed,LSTM
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import cv2
import re
import numpy as np
import os
import time
import json
from tensorflow.keras.applications.inception_v3 import  InceptionV3,preprocess_input
from tqdm import tqdm, trange
import math
from utils import batch_generator
# from NIC import model,greedy_inference_model,image_dense_lstm,text_emb_lstm
from pickle import dump, load

In [2]:
fp3= "data/captions_train2014.json"
with open(fp3,'r') as f3:
    mscoco_data= json.load(f3)
    

all_img_name_vector=[]
all_captions=[]
PATH_DIR= '/home/yiyang/project/caption/caption'
PATH_TRAIN=PATH_DIR + '/data/train2014/'
for annot in mscoco_data['annotations']:
    caption =  annot['caption']
   
    caption = caption.rstrip()
    caption = caption.lower()
    caption = [x.replace(".", "") for x in caption]
    caption = "".join(caption)
    caption = caption.rstrip()
    caption = "".join(caption)
    
    image_id = annot['image_id']
    full_coco_image_path = PATH_TRAIN + 'COCO_train2014_' + '%012d.jpg' % (image_id)

    all_img_name_vector.append(full_coco_image_path)
    all_captions.append(caption)
#     
# train_captions, img_name_vector = shuffle(all_captions,
#                                           all_img_name_vector,
#                                           random_state=1)
num_examples = 400000
train_captions = all_captions[:num_examples]
image_vec = all_img_name_vector[:num_examples]



In [3]:
fp4= "sentence_dict2.json"
with open(fp4,"r") as f4 :
    sentiment_dict= json.load(f4)
    

sentiment_vec=[]
caption_vec=[]
error=0
for i in range(num_examples):
    cap= train_captions[i]
    if cap in sentiment_dict.keys():
        senti= sentiment_dict[cap]
        
    else:
        senti=0
        error+=1
        # print(cap)
    cap= '<start> ' + cap +' <end>'
    caption_vec.append(cap)
    sentiment_vec.append(senti)
    
print(error)




25687


In [4]:
num_examples=400000
image_name_train= image_vec[:num_examples]
senti_train= sentiment_vec[:num_examples]
caption_train= caption_vec[:num_examples]
print(caption_train[0])


<start> a very clean and well decorated empty bathroom <end>


In [5]:
def load_image(image_path):
    img= cv2.imread(image_path)
    img= cv2.resize(img,(299,299))
    img = (img*1.0)/255
    img = img-0.5 
    img = img*2
    return img






In [6]:
fp2="data/senticap_dataset.json"

with open(fp2,'r') as f2:
    senticap_data= json.load(f2)
senticap_list=[]
senticap_list_raw=[]
l=0
for img_obj in senticap_data['images']:
    # if (l>20):
    #     break
    # l+=1
    img_name= img_obj['filename']
    for sent in img_obj['sentences']:
        sentiment= sent['sentiment']
        raw= sent['raw'].lower()
        w0= raw.rstrip()
        w0 = [x.replace(".", "") for x in w0]
        w0= [x.replace(",","") for x in w0]
        w0= [x.replace("'","") for x in w0]
        w0= [x.replace("?","") for x in w0]
        w0="".join(w0)
        senticap_list.append((img_name,sentiment,w0))
        senticap_list_raw.append((img_name,w0))


        

In [7]:
"""read from output of nlp treebank"""

fname="new_out.txt"
i=0
l1=10000
senti_score=[]
senti_avg=[]
error=0
senti_var=[]
train_caption=[]
senti_cap_dict={}
with open(fname) as f1:
    while(i<l1):
        w1= f1.readline().strip()
        if not w1:
            break
        w2= f1.readline().strip()
        w1 = [x.replace(".", "") for x in w1]
        w1= [x.replace(",","") for x in w1]
        w1= [x.replace("'","") for x in w1]
        w1= [x.replace("?","") for x in w1]
        w1= "".join(x for x in w1)
        w1= w1.rstrip()
        w1= "".join(x for x in w1)

        if w2.islower():
            w1=w2
            w2=f1.readline().strip()
        a= 0
        if w2=="Negative":
            a= -1
        elif w2=="Positive":
            a= 1
        elif w2=="Very negative":
            a= -1
        elif w2=="Very positive":
            a= 1
        elif w2=="Neutral":
            a= 0
        else:
            a=0
            error+=1
            print(w2)
        senti_score.append(a)
        train_caption.append(w1)
        senti_cap_dict[w1]=a
        i+=1
# print(senti_score[0:500] )
print(len(train_caption),"train-cap_length")
print(len(senti_score),"train-senti_score")


" .
8861 train-cap_length
8861 train-senti_score


In [8]:
PATH_DIR= '/home/yiyang/project/caption/caption'
PATH= PATH_DIR+'/data/val2014/'
# image_vec=[]
# sentiment_vec=[]
# caption_vec=[]

for item in senticap_list:
    img= item[0]
    if item[1]==1:
        senti=1
    elif item[1]==0:
        senti=-1
    else:
        senti=0

    caption= item[2]
    # print(caption+"#")
    if caption in senti_cap_dict.keys():
        new_senti= senti_cap_dict[caption]
    else:
        new_senti= 0
    img= PATH + img 
    caption= '<start> ' + caption + ' <end>'
    image_vec.append(img)
    sentiment_vec.append(new_senti)
    caption_vec.append(caption)
    
print(len(image_vec),"length of combined dataset")
print(len(caption_vec))
print(len(sentiment_vec))

408848 length of combined dataset
408848
408848


In [9]:
units= 512
embedding_size=512
BATCH_SIZE=80
vocab_size=5000
max_length=20
num_examples=200000
reg=1e-4

In [10]:
t=408848
image_name_train= image_vec[t-num_examples:]
senti_train= sentiment_vec[t-num_examples:]
caption_train= caption_vec[t-num_examples:]


In [11]:
vocab_size=5000
top_k = vocab_size-1
tokenizer = keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(caption_vec)
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'


train_seqs = tokenizer.texts_to_sequences(caption_train)
cap_vector_train = keras.preprocessing.sequence.pad_sequences(train_seqs,dtype='int32', padding='post',truncating='post',maxlen=max_length+1,value=0)

print(caption_train[0])
print(cap_vector_train[0])



<start> a close up of bananas growing on a cliff face <end>
[   3    2  132   36    7  209 1133    5    2 1575  355    4    0    0
    0    0    0    0    0    0    0]


In [12]:
print(len(image_name_train))

200000


In [16]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [17]:
weights_path = 'data/image_net.h5'

In [30]:
class Image_model(tf.keras.Model):
    
    def __init__(self):
        super(Image_model, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.image_model= InceptionV3(include_top=True,weights=None)
        self.image_model.load_weights(weights_path)
        for layer in self.image_model.layers[:312]:
            layer.trainable = False
        self.model_input= self.image_model.input
        self.model_output= self.image_model.get_layer('avg_pool').output
        self.new_model=tf.keras.Model(self.model_input,self.model_output)
    def call(self,x):
        output= self.new_model(x)
        return output
    

In [31]:
def load_image2(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path


In [32]:
image_features_extract_model = Image_model()


In [33]:
encode_train = sorted(set(image_vec))

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image2, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)
for img, path in tqdm(image_dataset):
    batch_features = image_features_extract_model.predict(x=img, steps=16)
    # batch_features = image_features_extract_model(img)
      # batch_features = tf.reshape(batch_features,
      #                             (batch_features.shape[0], -1, batch_features.shape[3]))
    batch_features=np.reshape(batch_features,(-1,batch_features.shape[1]))
    # break
    # np.save(path,batch_features)

    for bf, p in tqdm(zip(batch_features, path)):
        path_of_feature = p.numpy().decode("utf-8")
        # path_of_feature = p
        # break
        np.save(path_of_feature, bf)
        # break
#         

0it [00:00, ?it/s]
16it [00:00, 3727.03it/s]
1it [00:03,  3.62s/it]
16it [00:00, 3807.60it/s]
2it [00:04,  2.70s/it]
16it [00:00, 3745.12it/s]
3it [00:04,  2.06s/it]
16it [00:00, 3905.77it/s]
4it [00:05,  1.61s/it]
16it [00:00, 3782.70it/s]
5it [00:05,  1.29s/it]
16it [00:00, 3850.19it/s]
6it [00:06,  1.08s/it]
16it [00:00, 3817.56it/s]
7it [00:07,  1.08it/s]
16it [00:00, 3747.84it/s]
8it [00:07,  1.22it/s]
16it [00:00, 3825.17it/s]
9it [00:08,  1.35it/s]
16it [00:00, 3853.73it/s]
10it [00:08,  1.45it/s]
16it [00:00, 3859.71it/s]
11it [00:09,  1.54it/s]
16it [00:00, 3680.42it/s]
12it [00:09,  1.61it/s]
16it [00:00, 3878.90it/s]
13it [00:10,  1.66it/s]
16it [00:00, 3543.61it/s]
14it [00:10,  1.70it/s]
16it [00:00, 3592.55it/s]
15it [00:11,  1.74it/s]
16it [00:00, 3582.58it/s]
16it [00:12,  1.76it/s]
16it [00:00, 3515.58it/s]
17it [00:12,  1.78it/s]
16it [00:00, 3952.93it/s]
18it [00:13,  1.78it/s]
16it [00:00, 3684.47it/s]
19it [00:13,  1.78it/s]
16it [00:00, 3517.42it/s]
20it [00:14,  

16it [00:00, 4078.33it/s]
163it [01:34,  1.80it/s]
16it [00:00, 3287.88it/s]
164it [01:35,  1.79it/s]
16it [00:00, 3943.17it/s]
165it [01:35,  1.78it/s]
16it [00:00, 3893.98it/s]
166it [01:36,  1.77it/s]
16it [00:00, 3770.16it/s]
167it [01:36,  1.77it/s]
16it [00:00, 3745.54it/s]
168it [01:37,  1.78it/s]
16it [00:00, 3657.56it/s]
169it [01:37,  1.79it/s]
16it [00:00, 3533.91it/s]
170it [01:38,  1.79it/s]
16it [00:00, 3540.62it/s]
171it [01:38,  1.79it/s]
16it [00:00, 3529.63it/s]
172it [01:39,  1.79it/s]
16it [00:00, 3801.34it/s]
173it [01:40,  1.79it/s]
16it [00:00, 3856.83it/s]
174it [01:40,  1.79it/s]
16it [00:00, 3832.60it/s]
175it [01:41,  1.79it/s]
16it [00:00, 3835.01it/s]
176it [01:41,  1.80it/s]
16it [00:00, 3941.32it/s]
177it [01:42,  1.79it/s]
16it [00:00, 3877.78it/s]
178it [01:42,  1.78it/s]
16it [00:00, 3808.46it/s]
179it [01:43,  1.79it/s]
16it [00:00, 3856.83it/s]
180it [01:43,  1.78it/s]
16it [00:00, 3827.79it/s]
181it [01:44,  1.78it/s]
16it [00:00, 4015.61it/s]
182it

323it [03:04,  1.77it/s]
16it [00:00, 3744.50it/s]
324it [03:05,  1.77it/s]
16it [00:00, 3923.12it/s]
325it [03:05,  1.76it/s]
16it [00:00, 3687.30it/s]
326it [03:06,  1.76it/s]
16it [00:00, 4021.87it/s]
327it [03:06,  1.78it/s]
16it [00:00, 3604.52it/s]
328it [03:07,  1.77it/s]
16it [00:00, 3754.97it/s]
329it [03:07,  1.77it/s]
16it [00:00, 3852.40it/s]
330it [03:08,  1.76it/s]
16it [00:00, 3772.28it/s]
331it [03:09,  1.77it/s]
16it [00:00, 3912.37it/s]
332it [03:09,  1.78it/s]
16it [00:00, 3808.03it/s]
333it [03:10,  1.78it/s]
16it [00:00, 3421.48it/s]
334it [03:10,  1.77it/s]
16it [00:00, 3553.55it/s]
335it [03:11,  1.77it/s]
16it [00:00, 3510.25it/s]
336it [03:11,  1.77it/s]
16it [00:00, 3822.12it/s]
337it [03:12,  1.76it/s]
16it [00:00, 3886.54it/s]
338it [03:13,  1.77it/s]
16it [00:00, 3840.28it/s]
339it [03:13,  1.78it/s]
16it [00:00, 3863.94it/s]
340it [03:14,  1.78it/s]
16it [00:00, 3863.94it/s]
341it [03:14,  1.78it/s]
16it [00:00, 3942.71it/s]
342it [03:15,  1.78it/s]
16it [

16it [00:00, 3717.53it/s]
484it [04:35,  1.77it/s]
16it [00:00, 3886.09it/s]
485it [04:35,  1.76it/s]
16it [00:00, 3742.41it/s]
486it [04:36,  1.77it/s]
16it [00:00, 3875.99it/s]
487it [04:37,  1.77it/s]
16it [00:00, 3206.04it/s]
488it [04:37,  1.78it/s]
16it [00:00, 3328.98it/s]
489it [04:38,  1.79it/s]
16it [00:00, 2298.09it/s]
490it [04:38,  1.78it/s]
16it [00:00, 3849.31it/s]
491it [04:39,  1.79it/s]
16it [00:00, 2134.91it/s]
492it [04:39,  1.79it/s]
16it [00:00, 3470.31it/s]
493it [04:40,  1.79it/s]
16it [00:00, 2138.52it/s]
494it [04:40,  1.79it/s]
16it [00:00, 3218.50it/s]
495it [04:41,  1.78it/s]
16it [00:00, 3277.28it/s]
496it [04:42,  1.78it/s]
16it [00:00, 3670.36it/s]
497it [04:42,  1.78it/s]
16it [00:00, 3674.78it/s]
498it [04:43,  1.78it/s]
16it [00:00, 3274.08it/s]
499it [04:43,  1.77it/s]
16it [00:00, 3346.91it/s]
500it [04:44,  1.76it/s]
16it [00:00, 3434.08it/s]
501it [04:44,  1.78it/s]
16it [00:00, 3445.54it/s]
502it [04:45,  1.77it/s]
16it [00:00, 2177.73it/s]
503it

644it [06:05,  1.79it/s]
16it [00:00, 3893.75it/s]
645it [06:06,  1.80it/s]
16it [00:00, 3498.17it/s]
646it [06:06,  1.78it/s]
16it [00:00, 3434.26it/s]
647it [06:07,  1.78it/s]
16it [00:00, 3914.19it/s]
648it [06:07,  1.79it/s]
16it [00:00, 3759.81it/s]
649it [06:08,  1.79it/s]
16it [00:00, 3823.43it/s]
650it [06:09,  1.78it/s]
16it [00:00, 3880.25it/s]
651it [06:09,  1.78it/s]
16it [00:00, 3814.95it/s]
652it [06:10,  1.78it/s]
16it [00:00, 3813.87it/s]
653it [06:10,  1.79it/s]
16it [00:00, 3527.03it/s]
654it [06:11,  1.79it/s]
16it [00:00, 3872.41it/s]
655it [06:11,  1.80it/s]
16it [00:00, 3826.27it/s]
656it [06:12,  1.79it/s]
16it [00:00, 3859.94it/s]
657it [06:12,  1.79it/s]
16it [00:00, 3936.93it/s]
658it [06:13,  1.78it/s]
16it [00:00, 3948.04it/s]
659it [06:14,  1.77it/s]
16it [00:00, 3898.73it/s]
660it [06:14,  1.78it/s]
16it [00:00, 3773.34it/s]
661it [06:15,  1.79it/s]
16it [00:00, 4004.59it/s]
662it [06:15,  1.79it/s]
16it [00:00, 3926.79it/s]
663it [06:16,  1.78it/s]
16it [

...

4273it [40:15,  1.78it/s]
16it [00:00, 3756.65it/s]
4274it [40:15,  1.76it/s]
16it [00:00, 3848.43it/s]
4275it [40:16,  1.76it/s]
16it [00:00, 3817.99it/s]
4276it [40:16,  1.77it/s]
16it [00:00, 3829.54it/s]
4277it [40:17,  1.78it/s]
16it [00:00, 3859.71it/s]
4278it [40:18,  1.77it/s]
16it [00:00, 3929.78it/s]
4279it [40:18,  1.78it/s]
16it [00:00, 3713.21it/s]
4280it [40:19,  1.78it/s]
16it [00:00, 3832.38it/s]
4281it [40:19,  1.78it/s]
16it [00:00, 3861.27it/s]
4282it [40:20,  1.78it/s]
16it [00:00, 3899.86it/s]
4283it [40:20,  1.78it/s]
16it [00:00, 3911.68it/s]
4284it [40:21,  1.77it/s]
16it [00:00, 3864.60it/s]
4285it [40:21,  1.78it/s]
16it [00:00, 3740.74it/s]
4286it [40:22,  1.79it/s]
16it [00:00, 3861.71it/s]
4287it [40:23,  1.77it/s]
16it [00:00, 3901.68it/s]
4288it [40:23,  1.77it/s]
16it [00:00, 3688.92it/s]
4289it [40:24,  1.77it/s]
16it [00:00, 3639.51it/s]
4290it [40:24,  1.78it/s]
16it [00:00, 3905.99it/s]
4291it [40:25,  1.78it/s]
16it [00:00, 3792.10it/s]
4292it [40:2

16it [00:00, 3755.18it/s]
4431it [41:44,  1.77it/s]
16it [00:00, 3801.99it/s]
4432it [41:45,  1.78it/s]
16it [00:00, 3837.20it/s]
4433it [41:45,  1.78it/s]
16it [00:00, 3935.54it/s]
4434it [41:46,  1.77it/s]
16it [00:00, 3846.22it/s]
4435it [41:46,  1.77it/s]
16it [00:00, 3875.54it/s]
4436it [41:47,  1.78it/s]
16it [00:00, 3879.35it/s]
4437it [41:47,  1.78it/s]
16it [00:00, 3873.75it/s]
4438it [41:48,  1.77it/s]
16it [00:00, 3823.65it/s]
4439it [41:48,  1.78it/s]
16it [00:00, 3922.66it/s]
4440it [41:49,  1.76it/s]
16it [00:00, 3858.38it/s]
4441it [41:50,  1.77it/s]
16it [00:00, 3833.26it/s]
4442it [41:50,  1.77it/s]
16it [00:00, 3813.87it/s]
4443it [41:51,  1.78it/s]
16it [00:00, 3930.01it/s]
4444it [41:51,  1.78it/s]
16it [00:00, 3908.50it/s]
4445it [41:52,  1.78it/s]
16it [00:00, 3778.87it/s]
4446it [41:52,  1.77it/s]
16it [00:00, 3796.18it/s]
4447it [41:53,  1.76it/s]
16it [00:00, 3752.03it/s]
4448it [41:54,  1.77it/s]
16it [00:00, 3832.60it/s]
4449it [41:54,  1.78it/s]
16it [00:00,

4588it [43:13,  1.72it/s]
16it [00:00, 3749.52it/s]
4589it [43:13,  1.73it/s]
16it [00:00, 3794.68it/s]
4590it [43:14,  1.73it/s]
16it [00:00, 3828.01it/s]
4591it [43:14,  1.72it/s]
16it [00:00, 3878.45it/s]
4592it [43:15,  1.73it/s]
16it [00:00, 3917.39it/s]
4593it [43:16,  1.73it/s]
16it [00:00, 3911.46it/s]
4594it [43:16,  1.73it/s]
16it [00:00, 3908.95it/s]
4595it [43:17,  1.75it/s]
16it [00:00, 3860.16it/s]
4596it [43:17,  1.75it/s]
16it [00:00, 3903.95it/s]
4597it [43:18,  1.76it/s]
16it [00:00, 3803.28it/s]
4598it [43:18,  1.77it/s]
16it [00:00, 3993.86it/s]
4599it [43:19,  1.77it/s]
16it [00:00, 3838.08it/s]
4600it [43:20,  1.76it/s]
16it [00:00, 3926.56it/s]
4601it [43:20,  1.76it/s]
16it [00:00, 3816.69it/s]
4602it [43:21,  1.77it/s]
16it [00:00, 3726.20it/s]
4603it [43:21,  1.77it/s]
16it [00:00, 3867.05it/s]
4604it [43:22,  1.76it/s]
16it [00:00, 3863.27it/s]
4605it [43:22,  1.76it/s]
16it [00:00, 3871.52it/s]
4606it [43:23,  1.75it/s]
16it [00:00, 3796.39it/s]
4607it [43:2

16it [00:00, 3862.16it/s]
4746it [44:42,  1.77it/s]
16it [00:00, 3299.52it/s]
4747it [44:43,  1.77it/s]
16it [00:00, 3297.25it/s]
4748it [44:43,  1.78it/s]
16it [00:00, 3822.78it/s]
4749it [44:44,  1.77it/s]
16it [00:00, 3867.50it/s]
4750it [44:44,  1.78it/s]
16it [00:00, 3745.12it/s]
4751it [44:45,  1.77it/s]
16it [00:00, 3705.62it/s]
4752it [44:45,  1.75it/s]
16it [00:00, 3698.68it/s]
4753it [44:46,  1.76it/s]
16it [00:00, 3751.82it/s]
4754it [44:47,  1.77it/s]
16it [00:00, 3685.88it/s]
4755it [44:47,  1.76it/s]
16it [00:00, 3799.40it/s]
4756it [44:48,  1.77it/s]
16it [00:00, 3907.13it/s]
4757it [44:48,  1.75it/s]
16it [00:00, 3808.89it/s]
4758it [44:49,  1.75it/s]
16it [00:00, 3723.93it/s]
4759it [44:49,  1.75it/s]
16it [00:00, 2606.47it/s]
4760it [44:50,  1.75it/s]
16it [00:00, 3825.83it/s]
4761it [44:51,  1.75it/s]
16it [00:00, 3867.50it/s]
4762it [44:51,  1.77it/s]
16it [00:00, 3919.45it/s]
4763it [44:52,  1.76it/s]
16it [00:00, 3845.12it/s]
4764it [44:52,  1.77it/s]
16it [00:00,

4903it [46:11,  1.76it/s]
16it [00:00, 3742.83it/s]
4904it [46:11,  1.77it/s]
16it [00:00, 3889.69it/s]
4905it [46:12,  1.77it/s]
16it [00:00, 3929.09it/s]
4906it [46:13,  1.77it/s]
16it [00:00, 3456.01it/s]
4907it [46:13,  1.78it/s]
16it [00:00, 3996.00it/s]
4908it [46:14,  1.78it/s]
16it [00:00, 3785.47it/s]
4909it [46:14,  1.77it/s]
16it [00:00, 3796.82it/s]
4910it [46:15,  1.76it/s]
16it [00:00, 3931.62it/s]
4911it [46:15,  1.74it/s]
16it [00:00, 3844.90it/s]
4912it [46:16,  1.76it/s]
16it [00:00, 3861.71it/s]
4913it [46:17,  1.76it/s]
16it [00:00, 3876.44it/s]
4914it [46:17,  1.76it/s]
16it [00:00, 3756.86it/s]
4915it [46:18,  1.77it/s]
16it [00:00, 3829.76it/s]
4916it [46:18,  1.78it/s]
16it [00:00, 3717.32it/s]
4917it [46:19,  1.78it/s]
16it [00:00, 3814.30it/s]
4918it [46:19,  1.78it/s]
16it [00:00, 3653.97it/s]
4919it [46:20,  1.79it/s]
16it [00:00, 3886.76it/s]
4920it [46:21,  1.77it/s]
16it [00:00, 3901.22it/s]
4921it [46:21,  1.76it/s]
16it [00:00, 3746.17it/s]
4922it [46:2

In [13]:
class RNN_train(tf.keras.Model):
    def __init__(self,vocab_size, max_len, units, reg, img_embedding_size):
        super(RNN_train, self).__init__()
        # self.image_model= Image_model()
        self.vocab_size=vocab_size
        self.max_len= max_len
        self.units=units
        self.reg= reg
        self.image_embedding_layer= Dense(img_embedding_size, use_bias=False, name = 'emb_img')
        self.seq_embedding_layer=Embedding(vocab_size, units, mask_zero = False, name = 'emb_text')
        self.BN_layer= BatchNormalization(name='batch_normalization')
        self.expand_dim= Lambda(lambda x : K.expand_dims(x, axis=1))
        # self.state_c= tf.zeros()
        self.lstm_layer=  LSTM(self.units, return_sequences = True, return_state = True, dropout=0.5, name = 'lstm')
        self.softmax_layer= Dense(vocab_size, activation='softmax')
        self.time_distribute= TimeDistributed(self.softmax_layer)
        self.argmax_layer=  Lambda(lambda x : (K.argmax(x)))
        
    def call(self, inputs):
        # image_input = Input(shape=(2048)
        # 
        # seq_input = Input(shape=(self.max_len,))
        # senti_input = Input(shape=(1,))
        # state_c = Input(shape=(self.units,))
        # state_h = Input(shape=(self.units,))
        
        image_input = inputs[0]
        
        seq_input = inputs[1]
        senti_input = inputs[2]
        state_c = inputs[3]
        state_h = inputs[4]
        # tf.print(senti_input)
        
        # x= CNN_encoder(image_input)
        # tf.print(x)
        x= Dropout(0.5)(image_input)
        x= self.image_embedding_layer(x)
        senti_input= K.cast(senti_input,dtype=tf.float32)
        # tf.print(senti_input)
        # x= Concatenate(axis=-1)([senti_input, x])
        x= self.BN_layer(x)
        x= self.expand_dim(x)
        
        x_seq= self.seq_embedding_layer(seq_input)
        # tf.print(x_seq.shape,"embed_seq")
        x_seq= Dropout(0.5)(x_seq)
        
        _, init_state_c, init_state_h = self.lstm_layer(x, initial_state=[state_c, state_h])
        
        out, _, _ = self.lstm_layer(x_seq, initial_state=[init_state_c, init_state_h])
        # tf.print(out.shape,"lstm_out")
        output= self.time_distribute(out)
        
        # train_model= Model(inputs=[image_input,seq_input,senti_input, state_c, state_h], outputs=output)
        return output
        
    def inference_call(self, inputs):
        image_input = inputs[0]
        
        seq_input = inputs[1]
        senti_input = inputs[2]
        state_c0 = inputs[3]
        state_h0 = inputs[4]
        x= image_input
        # tf.print(x)

        x= self.image_embedding_layer(x)
        # tf.print(x.shape,"emb_img")
        x_senti= K.cast(senti_input,dtype=tf.float32)
        x_senti= tf.expand_dims(x_senti,axis=-1)
        # tf.print(senti_input)
        # x= Concatenate(axis=-1)([x_senti, x])
        x= self.BN_layer(x)
        x= self.expand_dim(x)
        # tf.print(x.shape,"img_input")
        x_seq= self.seq_embedding_layer(seq_input)
        x_seq= tf.expand_dims(x_seq,axis=1)
        # tf.print(x_seq.shape,"emb_seq")
        # tf.print(state_c0.shape,"c0")
        o, state_c, state_h = self.lstm_layer(x, initial_state=[state_c0, state_h0])
        # tf.print(state_c.shape,"init_c")
        # tf.print(o.shape,"img_out_shape")
        outputs = []

        for i in range(self.max_len):

            out, state_c, state_h = self.lstm_layer(x_seq, initial_state=[state_c,state_h])
            # tf.print(state_c.shape,"c")
            
            output = self.softmax_layer(state_c)
            # tf.print(output.shape,"softmax_output_shape")
            # outputs.append(output)
            x_seq = self.argmax_layer(output)
            outputs.append(x_seq)
            x_seq= K.expand_dims(x_seq)
            # tf.print(x_seq.shape,"argmax")
            x_seq = self.seq_embedding_layer(x_seq)
            # tf.print(x_seq.shape,"new_seq")
            
        return outputs
        
        
        


In [14]:
reg=1e-4
decay=0
lr=0.0001
initial_epoch=0
embedding_size=512

# NIC_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr = lr, decay=decay), metrics=['accuracy'])

rnn_train_model= RNN_train(vocab_size,max_length,units,reg,embedding_size)
rnn_train_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr = lr, decay=decay), metrics=['accuracy'])

In [22]:

# print(caption_train[0].shape)
start=0
train_num=200000
generator_train= batch_generator(BATCH_SIZE,vocab_size,train_num,start,image_name_train,senti_train,cap_vector_train,)


In [23]:
start2=0
train_num2=2000
generator_dev= batch_generator(BATCH_SIZE,vocab_size,train_num2,start2,image_name_train,senti_train,cap_vector_train,)


In [35]:
# tf.config.experimental_run_functions_eagerly(False)
BATCH_SIZE=80
# rnn_train_model.load_weights("models/5kmodel0_200kdata.h5")
rnn_train_model.fit_generator(generator_train, steps_per_epoch=train_num//BATCH_SIZE,
                        epochs=11,
                        verbose=1, 
                        initial_epoch = 10)

# rnn_train_model.save_weights("models/5kmodel0_200kdata.h5")

Epoch 11/11
   1/2500 [..............................] - ETA: 6:36 - loss: 1.2755 - accuracy: 0.7144

   2/2500 [..............................] - ETA: 7:12 - loss: 1.3285 - accuracy: 0.7050

   3/2500 [..............................] - ETA: 7:11 - loss: 1.3226 - accuracy: 0.7075

   4/2500 [..............................] - ETA: 6:45 - loss: 1.3360 - accuracy: 0.7031

   5/2500 [..............................] - ETA: 7:35 - loss: 1.3289 - accuracy: 0.7035

...

2496/2500 [============================>.] - ETA: 0s - loss: 1.2188 - accuracy: 0.7180

2497/2500 [============================>.] - ETA: 0s - loss: 1.2188 - accuracy: 0.7180

2498/2500 [============================>.] - ETA: 0s - loss: 1.2188 - accuracy: 0.7180

2499/2500 [============================>.] - ETA: 0s - loss: 1.2188 - accuracy: 0.7180

2500/2500 [==============================] - 357s 143ms/step - loss: 1.2188 - accuracy: 0.7180


In [29]:
rnn_train_model.save_weights("models/5kmodel0_200kdata.h5")
# rnn_train_model.load_weights("models/12kmodel0.h5")


In [17]:

# rnn_train_model.load_weights("models/5kmodel0.h5")
start=0
train_num=20000
generator_train= batch_generator(BATCH_SIZE,vocab_size,train_num,start,image_name_train,senti_train,cap_vector_train,)

rnn_train_model.fit_generator(generator_train, steps_per_epoch=train_num//BATCH_SIZE,
                        epochs=1,
                        verbose=1, 
                        initial_epoch = 0)
rnn_train_model.load_weights("models/5kmodel0_200kdata.h5")
# rnn_train_model.save_weights("models/5kmodel0.h5")




  1/250 [..............................] - ETA: 14:40 - loss: 8.5187 - accuracy: 0.0000e+00

  2/250 [..............................] - ETA: 7:32 - loss: 8.5159 - accuracy: 0.0000e+00 

  3/250 [..............................] - ETA: 5:08 - loss: 8.5140 - accuracy: 0.0018    

  4/250 [..............................] - ETA: 3:56 - loss: 8.5114 - accuracy: 0.0140

  5/250 [..............................] - ETA: 3:13 - loss: 8.5089 - accuracy: 0.0313

...

246/250 [============================>.] - ETA: 0s - loss: 4.6623 - accuracy: 0.3860

247/250 [============================>.] - ETA: 0s - loss: 4.6565 - accuracy: 0.3862

248/250 [============================>.] - ETA: 0s - loss: 4.6512 - accuracy: 0.3863

249/250 [============================>.] - ETA: 0s - loss: 4.6463 - accuracy: 0.3865

250/250 [==============================] - 27s 108ms/step - loss: 4.6411 - accuracy: 0.3868


In [36]:
test_imgs=[]
num_test=2000
start=0
for j in tqdm(range(start,start+num_test)):
    img= np.load( image_name_train[j]+'.npy')
    test_imgs.append(img)
test_imgs= np.array(test_imgs,dtype=np.int)
test_sentis= np.array(senti_train[start:start+num_test],dtype=np.float32)
test_sequences= np.array(cap_vector_train[start:start+num_test])
# test_seq= np.ones(shape=(500,1))
# test_seq= 3* test_seq
# test_out=[]
print(test_imgs.shape)
tf.print(test_imgs[0].shape)
tf.print(test_imgs[0])

  0%|          | 0/2000 [00:00<?, ?it/s]

 34%|███▍      | 683/2000 [00:00<00:00, 6827.19it/s]

 69%|██████▉   | 1381/2000 [00:00<00:00, 6871.79it/s]

100%|██████████| 2000/2000 [00:00<00:00, 6697.86it/s]

(2000, 2048)
(2048,)


array([0, 0, 1, ..., 0, 0, 0])


In [37]:
N = test_imgs.shape[0]

startseq = np.repeat([tokenizer.word_index['<start>']], N)
a0 = np.zeros([N, units],dtype=np.float32)
c0 = np.zeros([N, units],dtype=np.float32)
# test_sentis= np.ones([num_test])
# test_sentis= - np.ones([num_test])
# test_sentis= np.zeros([num_test])

# output dims: [32, N, 7378]
test_num=N
img_input= tf.convert_to_tensor(test_imgs[:test_num])
seq_input= tf.convert_to_tensor(startseq[:test_num])
senti_input= tf.convert_to_tensor(test_sentis[:test_num])
state_a0= tf.convert_to_tensor(a0[:test_num])
state_c0= tf.convert_to_tensor(c0[:test_num])
y_preds= rnn_train_model.inference_call([img_input,seq_input,senti_input,state_a0,state_c0])
y_preds= np.array(y_preds)

y_preds = np.transpose(y_preds, axes = [1,0])
# print(y_preds)
# y_preds = np.array(rnn_train_model.inference_call([test_imgs[:80], startseq[:80],test_sentis[:80], a0[:80], c0[:80]]))
# y_preds = RNN_inf(test_imgs[:10], startseq[:10],test_sentis[:10], a0[:10], c0[:10])
    
# output dims: [N, 32, 7378]
# y_preds = np.transpose(y_preds, axes = [1,0,2])
# 
# sequences = np.argmax(y_preds, axis = -1)
sequences=y_preds
sents = tokenizer.sequences_to_texts(sequences)
# print(sents[450:])
# 
# # if post_process:
#     # post processing: 'endseq'
sents_pp = []
for sent in sents:
    if '<end>' in sent.split():
        words = sent.split()
        sents_pp.append(' '.join(words[:words.index('<end>')]))
    else:
        sents_pp.append(sent)
sents = sents_pp


In [38]:
print(sents[:10])

['a bunch of bananas growing from a tree', 'a young boy is playing frisbee in a field', 'a beautiful blue and yellow train is in the grass', 'a young boy is playing frisbee in the park', 'a man riding skis down a snow covered slope', 'a man riding a skateboard down a road', 'a dog is running through the grass with a frisbee in its mouth', 'a close up of a very large green and yellow lighthouse', 'a group of stupid people standing around a field', 'a large tower with a clock on it']


In [39]:
# f1='output/gen_data.json'
# f2='output/gen_data.txt'
gen_data={}
gen_data['annotations']=[]
ref_data={}
ref_data['annotations']=[]
start=0
for i in trange(test_num):
    img_id= image_name_train[start+i]
    gen_cap= sents[i]
    ref_cap= caption_train[start+i]
    ref_cap= ref_cap.replace("<start>","")
    ref_cap= ref_cap.replace("<end>","")
    ref_cap= ref_cap.strip()
    ref_cap= "".join(ref_cap)
    gen_data['annotations'].append({'image_id':img_id,'caption':gen_cap})
    
    ref_data['annotations'].append({'image_id':img_id,'caption':ref_cap})
    
with open("output/gen_data.json", "w") as f2:
    json.dump(gen_data, f2)
    
with open("output/ref_data.json", "w") as f3:
    json.dump(ref_data, f3)

with open("output/gen_data_ori.txt","w") as f4:
    for item in gen_data["annotations"]:
        cap= item['caption']
        f4.write("%s .\n" % cap)

  0%|          | 0/2000 [00:00<?, ?it/s]

100%|██████████| 2000/2000 [00:00<00:00, 237254.52it/s]

In [ ]:

img_num=9
 
print(image_name_train[img_num])
# img=load_image(image_name_train[0])
img=cv2.imread(image_name_train[img_num]) 
print(caption_train[img_num])
print(senti_train[img_num])
# img=cv2.imshow('image',img)
cv2.imshow('ImageWindow', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

 


In [66]:
# # @tf.function
# def RNN_inf(image_input, seq_input, senti_input, state_c, state_h):
#     # image_input = Input(2048,)        
#     # seq_input = Input(1,)
#     # senti_input = Input(1,)
#     # state_c = Input(units)
#     # state_h = Input(units)
#     # tf.print(senti_input)
#     
#     # x= image_input
#     # tf.print(x.shape)
#     # tf.print(image_input.dtype)
#     # tf.print(seq_input.dtype)
#     # tf.print(senti_input.dtype)
#     # tf.print(state_c.dtype)
#     # tf.print(state_h.dtype)
#     
#     x= rnn_train_model.image_embedding_layer(image_input)
#     # tf.print(x.shape)
#     x_senti= K.cast(senti_input,dtype=tf.float32)
#     x_senti= tf.expand_dims(x_senti,axis=-1)
#     # tf.print(senti_input.shape,"senti_input_expand")
#     x= Concatenate(axis=-1)([x_senti, x])
#     # tf.print(x.shape)
#     x= rnn_train_model.BN_layer(x)
#     x= rnn_train_model.expand_dim(x)
#     # tf.print(seq_input.shape,"seq_input")
#     x_seq= tf.expand_dims(seq_input,axis=-1)
#     x_seq= rnn_train_model.seq_embedding_layer(x_seq)
#     
#     # tf.print(x_seq.shape,"embed seq")
#     _, init_state_c, init_state_h = rnn_train_model.lstm_layer(x, initial_state=[state_c, state_h])
#     # tf.print(init_state_c.shape,"state_c")
#     # out, _, _ = self.lstm_layer(x_seq, initial_state=[init_state_c, init_state_h])
#     # output= self.time_distribute(out)
#     
#     outputs = []
#     argmax_layer=  Lambda(lambda x : K.argmax(x))
#     
#     for i in range(rnn_train_model.max_len):
# 
#         out, state_c, state_h = rnn_train_model.lstm_layer(x_seq, initial_state=[init_state_c,init_state_h])
#         tf.print(out.shape,"lstm_out")
#         tf.print(state_c.shape,"state_c")
#         init_state_c= state_c 
#         init_state_h= state_h
#         output = rnn_train_model.softmax_layer(out)
#         # tf.print(output.shape,"softmax output")
#         outputs.append(output)
#         x_seq = argmax_layer(output)
#         # tf.print(x_seq.shape)
#         x_seq = rnn_train_model.seq_embedding_layer(x_seq)
#         # tf.print(x_seq.shape)
#             
#     return outputs